In [2]:
%%time
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_palette("afmhot")

sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth":1.5})
sns.set_style("whitegrid")
import pylab
pylab.rcParams['figure.figsize'] = (15, 8)
plt.style.use('dark_background')


import tensorflow as tf
print("# GPUs: ", len(tf.config.experimental.list_physical_devices("GPU")))
import numpy as np
import tensorflow.keras as tfk
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Dense, Activation, Dropout, Input, InputLayer
from tensorflow.keras.optimizers import SGD

import tensorflow.keras.datasets.mnist as mnist
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
data = mnist.load_data()


# GPUs:  1
CPU times: total: 219 ms
Wall time: 224 ms


In [3]:
(X_train, y_train),(X_test, y_test) = data
np.random.seed(42)

X_train, X_test = X_train / 255.0, X_test / 255.0

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)

NUM_CLASSES = 10

X_grid, X_junk, y_grid, y_junk = train_test_split(X_train, y_train, test_size=0.90, random_state=42)


INPUT_DIM = X_grid.shape[1]



learning_rate = [0.0001, 0.001, 0.01, 0.1]
momentum = [0.3, 0.5, 0.7, 0.9]



BATCH_SIZE = 100
EPOCHS = 5



In [4]:
X_grid.shape

(6000, 784)

# <font color = 'lime'> ANN Hyperparameter Tuning MANUAL:</font>

# <font color = 'lime'> ANN CV:</font>

# <font color = 'slateblue'> Cross Validation Complete

In [92]:
%%time
results = {}

def ANN2(LR=0.01,MOMENT=0.1,X=X_grid,y=y_grid,i=0):    
    model = Sequential()
    model.add(Dense(64, activation='relu', kernel_initializer='uniform',input_dim = INPUT_DIM))
    model.add(Dropout(0.1))
    model.add(Dense(64, kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(NUM_CLASSES, kernel_initializer='uniform',activation='softmax'))

    optimizer = SGD(learning_rate=LR, momentum = MOMENT)
    model.compile(loss='sparse_categorical_crossentropy',optimizer = optimizer, metrics=['acc'])
    
    # print(f'LR_{LR}__MOM{MOMENT}_{i}')
    history = model.fit(X, y,epochs=EPOCHS,batch_size = BATCH_SIZE,verbose=0)
    # print("")
    evaluation = model.evaluate(X_test,y_test,verbose=0)
    # print("")
    results[f'LR_{LR}__MOM{MOMENT}'] = evaluation[1]
    return evaluation[1]
    # pd.DataFrame(history.history).plot()
    # plt.title(f"LR_{LR}__MOM{MOMENT}")
    # plt.show();

CPU times: total: 0 ns
Wall time: 0 ns


# <font color = 'lime'> Cross Validation && HP-GS

In [87]:
learning_rate = [0.0001, 0.001, 0.01, 0.1]
momentum = [0.3, 0.5, 0.7, 0.9]

display(learning_rate)
display(momentum)

[0.0001, 0.001, 0.01, 0.1]

[0.3, 0.5, 0.7, 0.9]

In [90]:
np.random.randint(1,999)

615

In [101]:
%%time
from sklearn.utils import shuffle
def cv_np(K_FOLD = 5, np_shape = X_grid.shape[0],SEED=np.random.randint(1,999)):  
    K_FOLD = K_FOLD
    SLICE_LEN = X_grid.shape[0]//K_FOLD
    # SEED = 415
    np0 = shuffle(X_grid,y_grid,random_state=SEED)

    COUNTER=0
    AGG_SCORE = []
    AGG_DICT = {}
    AGG_DICT_SIMPLE = {}
    
    for lr in learning_rate:
        for mom in momentum:            
            for x in range(K_FOLD):
                COUNTER+=1
                i = (x*SLICE_LEN)
                j = ((x+1)*SLICE_LEN)
                # print(np0[0][i:j]) # slice iteration for X
                X_cv=np0[0][i:j]

                y_cv=np0[1][i:j]

                SCORE = ANN2(lr, mom, X_cv,y_cv)
                # AGG_SCORE.append(SCORE)
                
                # AGG_DICT[f"LR{lr}__M{mom}_{x}"] = SCORE
                
                if f"LR{lr}__M{mom}" not in AGG_DICT_SIMPLE.keys():
                    AGG_DICT_SIMPLE[f"LR{lr}__M{mom}"] = []
                else:
                    pass
                AGG_DICT_SIMPLE[f"LR{lr}__M{mom}"].append(SCORE)
                




    print("\n")
    print(COUNTER)
    return AGG_SCORE, AGG_DICT,AGG_DICT_SIMPLE

CPU times: total: 0 ns
Wall time: 0 ns


# <font color = 'lime'> Cross Validation && HP-GS @ FULL `X_train`

In [135]:
392/8

49.0

In [136]:
%%time
results = {}

def ANN3(LR=0.01,MOMENT=0.1,X=X_train,y=y_train,i=0):    
    model = Sequential()
    model.add(Dense(784, activation='relu', kernel_initializer='uniform',input_dim = INPUT_DIM))
    model.add(Dropout(0.25))
    
    model.add(Dense(392, kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(196, kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(98, kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(49, kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Dense(NUM_CLASSES, kernel_initializer='uniform',activation='softmax'))

    optimizer = SGD(learning_rate=LR, momentum = MOMENT)
    model.compile(loss='sparse_categorical_crossentropy',optimizer = optimizer, metrics=['acc'])
    
    # print(f'LR_{LR}__MOM{MOMENT}_{i}')
    history = model.fit(X, y,epochs=EPOCHS,batch_size = BATCH_SIZE,verbose=0)
    # print("")
    evaluation = model.evaluate(X_test,y_test,verbose=0)
    # print("")
    results[f'LR_{LR}__MOM{MOMENT}'] = evaluation[1]
    return evaluation[1]
    # pd.DataFrame(history.history).plot()
    # plt.title(f"LR_{LR}__MOM{MOMENT}")
    # plt.show();

CPU times: total: 0 ns
Wall time: 0 ns


In [138]:
%%time
from sklearn.utils import shuffle
def cv_np(K_FOLD = 5, np_shape = X_train.shape[0],SEED=np.random.randint(1,999)):  
    K_FOLD = K_FOLD
    SLICE_LEN = X_train.shape[0]//K_FOLD
    # SEED = 415
    np0 = shuffle(X_train,y_train,random_state=SEED)

    COUNTER=0
    AGG_SCORE = []
    AGG_DICT = {}
    AGG_DICT_SIMPLE = {}
    
    for lr in learning_rate:
        for mom in momentum:            
            for x in range(K_FOLD):
                COUNTER+=1
                i = (x*SLICE_LEN)
                j = ((x+1)*SLICE_LEN)
                # print(np0[0][i:j]) # slice iteration for X
                X_cv=np0[0][i:j]

                y_cv=np0[1][i:j]

                SCORE = ANN3(lr, mom, X_cv,y_cv)
                # AGG_SCORE.append(SCORE)
                
                # AGG_DICT[f"LR{lr}__M{mom}_{x}"] = SCORE
                
                if f"LR{lr}__M{mom}" not in AGG_DICT_SIMPLE.keys():
                    AGG_DICT_SIMPLE[f"LR{lr}__M{mom}"] = []
                else:
                    pass
                AGG_DICT_SIMPLE[f"LR{lr}__M{mom}"].append(SCORE)
                




    print("\n")
    print(COUNTER)
    return AGG_SCORE, AGG_DICT,AGG_DICT_SIMPLE

CPU times: total: 0 ns
Wall time: 0 ns


In [139]:
%%time
final100 = cv_np(5)



80
CPU times: total: 5min 27s
Wall time: 4min 25s


In [140]:
pd.DataFrame(final100[-1])

,LR0.0001__M0.3,LR0.0001__M0.5,LR0.0001__M0.7,LR0.0001__M0.9,LR0.001__M0.3,LR0.001__M0.5,LR0.001__M0.7,LR0.001__M0.9,LR0.01__M0.3,LR0.01__M0.5,LR0.01__M0.7,LR0.01__M0.9,LR0.1__M0.3,LR0.1__M0.5,LR0.1__M0.7,LR0.1__M0.9
0,0.1230,0.1138,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.5231,0.8649,0.9220
1,0.1575,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1161,0.4413,0.8939,0.9368
2,0.1135,0.1135,0.1337,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1549,0.4122,0.9064,0.9131
3,0.1145,0.0976,0.1006,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.4351,0.8791,0.9311
4,0.1172,0.1697,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.1135,0.2328,0.3016,0.8854,0.9332


In [141]:
pd.DataFrame(final100[-1]).mean()

LR0.0001__M0.3    0.12514
LR0.0001__M0.5    0.12162
LR0.0001__M0.7    0.11496
LR0.0001__M0.9    0.11350
LR0.001__M0.3     0.11350
LR0.001__M0.5     0.11350
LR0.001__M0.7     0.11350
LR0.001__M0.9     0.11350
LR0.01__M0.3      0.11350
LR0.01__M0.5      0.11350
LR0.01__M0.7      0.11350
LR0.01__M0.9      0.11350
LR0.1__M0.3       0.14616
LR0.1__M0.5       0.42266
LR0.1__M0.7       0.88594
LR0.1__M0.9       0.92724
dtype: float64

# SGD @ full 60,000
    LR0.0001__M0.3    0.12514
    LR0.0001__M0.5    0.12162
    LR0.0001__M0.7    0.11496
    LR0.0001__M0.9    0.11350
    LR0.001__M0.3     0.11350
    LR0.001__M0.5     0.11350
    LR0.001__M0.7     0.11350
    LR0.001__M0.9     0.11350
    LR0.01__M0.3      0.11350
    LR0.01__M0.5      0.11350
    LR0.01__M0.7      0.11350
    LR0.01__M0.9      0.11350
    LR0.1__M0.3       0.14616
    LR0.1__M0.5       0.42266
    LR0.1__M0.7       0.88594
    LR0.1__M0.9       0.92724
    dtype: float64

In [125]:
X_train.shape

(60000, 784)

In [104]:
# @6000 obs

# 80
# CPU times: total: 1min 52s
# Wall time: 1min 28s


# 80
# CPU times: total: 1min 50s
# Wall time: 1min 26s

# 80
# CPU times: total: 1min 53s
# Wall time: 1min 27s



# @ 60000 obs
# 80
# CPU times: total: 3min 54s
# Wall time: 2min 58s

In [106]:
pd.DataFrame(final2[-1]).mean()

LR0.0001__M0.3    0.10252
LR0.0001__M0.5    0.10026
LR0.0001__M0.7    0.10108
LR0.0001__M0.9    0.11408
LR0.001__M0.3     0.09240
LR0.001__M0.5     0.09936
LR0.001__M0.7     0.08668
LR0.001__M0.9     0.11820
LR0.01__M0.3      0.12278
LR0.01__M0.5      0.12670
LR0.01__M0.7      0.14312
LR0.01__M0.9      0.14574
LR0.1__M0.3       0.18116
LR0.1__M0.5       0.27790
LR0.1__M0.7       0.53914
LR0.1__M0.9       0.78588
dtype: float64

    LR0.0001__M0.3    0.08854
    LR0.0001__M0.5    0.08790
    LR0.0001__M0.7    0.11222
    LR0.0001__M0.9    0.11600
    LR0.001__M0.3     0.09048
    LR0.001__M0.5     0.09236
    LR0.001__M0.7     0.13160
    LR0.001__M0.9     0.13804
    LR0.01__M0.3      0.12514
    LR0.01__M0.5      0.14140
    LR0.01__M0.7      0.13324
    LR0.01__M0.9      0.17660
    LR0.1__M0.3       0.21770
    LR0.1__M0.5       0.30090
    LR0.1__M0.7       0.55796
    LR0.1__M0.9       0.78242
    dtype: float64